<a href="https://colab.research.google.com/github/YMGYM/google_colab_files/blob/master/NLP_With_Deep_Learning_Prcatice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro
이것은 https://wikidocs.net/24996 의 실습입니다

# import dataset

In [1]:
import pandas as pd
import urllib3
import zipfile
import shutil
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [3]:
http = urllib3.PoolManager()
url = 'http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)

with http.request("GET", url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:
  shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
  zip_ref.extractall(path)

In [11]:
lines = pd.read_csv("fra.txt", names=['src', 'tar', 'copy'], sep='\t')
len(lines)

178009

In [12]:
lines

,src,tar,copy
0,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
2,Hi.,Salut.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
3,Run!,Cours !,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
4,Run!,Courez !,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
...,...,...,...
178004,"Top-down economics never works, said Obama. ""T...","« L'économie en partant du haut vers le bas, ç...",CC-BY 2.0 (France) Attribution: tatoeba.org #1...
178005,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
178006,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
178007,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


In [13]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000]

In [15]:
lines.sample(10)

,src,tar
16202,School is boring.,"L'école, c'est ennuyeux !"
38764,Tom won't believe it.,Tom n'y croira pas.
36634,I'm not quitting now.,Je ne vais pas abandonner maintenant.
57864,There are a lot of rats.,Il y a beaucoup de rats.
33487,You need to move on.,Il te faut avancer.
26345,The wind blew hard.,Le vent soufflait fort.
32208,This is one of mine.,C'est l'une des miennes.
43712,Listen to your mother.,Écoute ta mère !
28429,Did I embarrass you?,Vous ai-je embarrassés ?
11721,I'm your friend.,Je suis votre amie.


# make CharSet
글자수 Set을 만들면 쉽게 중복을 안 셀수 있다...

In [16]:
src_vocab = set()

for line in lines.src: # 1줄씩 읽기
  for char in line: # 1개씩 읽기
    src_vocab.add(char)

In [17]:
src_vocab

{' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é',
 '’',
 '€'}

In [19]:
tar_vocab = set()
for line in lines.tar:
  for char in line:
    tar_vocab.add(char)

In [21]:
print(len(src_vocab ) + 1)
print(len(tar_vocab ) + 1)

79
104


# 글자단위 인코딩

In [22]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])

In [23]:
src_to_index

{' ': 7,
 '!': 54,
 '"': 13,
 '$': 28,
 '%': 72,
 '&': 70,
 "'": 46,
 ',': 60,
 '-': 74,
 '.': 21,
 '/': 38,
 '0': 44,
 '1': 59,
 '2': 61,
 '3': 51,
 '4': 23,
 '5': 22,
 '6': 77,
 '7': 48,
 '8': 49,
 '9': 15,
 ':': 73,
 '?': 27,
 'A': 31,
 'B': 36,
 'C': 71,
 'D': 65,
 'E': 69,
 'F': 39,
 'G': 35,
 'H': 53,
 'I': 64,
 'J': 18,
 'K': 24,
 'L': 33,
 'M': 43,
 'N': 62,
 'O': 67,
 'P': 11,
 'Q': 66,
 'R': 37,
 'S': 76,
 'T': 68,
 'U': 5,
 'V': 9,
 'W': 75,
 'X': 12,
 'Y': 41,
 'Z': 8,
 'a': 3,
 'b': 63,
 'c': 16,
 'd': 42,
 'e': 40,
 'f': 58,
 'g': 1,
 'h': 26,
 'i': 57,
 'j': 47,
 'k': 2,
 'l': 56,
 'm': 45,
 'n': 10,
 'o': 78,
 'p': 32,
 'q': 14,
 'r': 55,
 's': 19,
 't': 30,
 'u': 29,
 'v': 17,
 'w': 6,
 'x': 52,
 'y': 20,
 'z': 34,
 'é': 4,
 '’': 25,
 '€': 50}